# Imbalanced_Learning

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from collections import Counter
from collections import defaultdict

In [ ]:
X_train = pd.read_csv("X_train.txt", header=None,  delim_whitespace=True)
y_train = pd.read_csv("y_train.txt", header=None, delim_whitespace=True )
X_test = pd.read_csv("X_test.txt", header=None, delim_whitespace=True )
y_test = pd.read_csv("y_test.txt", header=None, delim_whitespace=True )
features = pd.read_csv("features.txt", header=None, delim_whitespace=True )
subject = pd.read_csv("subject_train.txt", header=None, delim_whitespace=True )

In [ ]:
feature = features
subjects = subject
feature.drop(0,inplace=True, axis=1)

In [ ]:
#inseriamo l'intestazione al dataset
lista=[]
feat_transpa = feature.transpose()
for i in range(561):
    lista.append(feat_transpa.iloc[0][i])
X_test.columns=lista    
X_train.columns=lista

In [ ]:
# eliminiamo le colonne che contengono la stima mad in quanto quasi uguale a dev.std
stringa="mad()"
for col in X_train.columns:
    if(stringa in col):
        X_train.drop(labels=col, axis=1, inplace=True)
for col in X_teast.columns:
    if(stringa in col):
        X_test.drop(labels=col, axis=1, inplace=True)        

In [ ]:
np.unique(y_train, return_counts=True)

## Undersampling

In [ ]:
#Inseriamo gli indici dei record delle classi da eliminare in una lista

In [ ]:
#inserisco in una lista gli indici delle classi che vorremmo togliere; TEST
prova_test1=[]
for i in range(len(y_test)):
    if( y_test.iloc[i][0]== 5 ):
        prova_test1.append(i)
#prova_test        

In [ ]:
#inserisco in una lista gli indici delle classi che vorremmo togliere; TEST
prova_test2=[]
for i in range(len(y_test)):
    if( y_test.iloc[i][0]== 2 ):
        prova_test2.append(i)
#prova_test        

In [ ]:
#inserisco in una lista gli indici delle classi che vorremmo togliere; TRAIN
prova_train1=[]
for i in range(len(y_train)):
    if(y_train.iloc[i][0]==5 ):
        prova_train1.append(i)
#prova_train     

In [ ]:
#inserisco in una lista gli indici delle classi che vorremmo togliere; TRAIN
prova_train2=[]
for i in range(len(y_train)):
    if(y_train.iloc[i][0]== 2):
        prova_train2.append(i)
#prova_train  

In [ ]:
y_test1 = pd.Series(prova_test1, copy=False) #5
y_test2 = pd.Series(prova_test2, copy=False) #2
len(y_test1),len(y_test2)

In [ ]:
y_train1 = pd.Series(prova_train1, copy=False)
y_train2 = pd.Series(prova_train2, copy=False)
len(y_train1), len(y_train2)

### leviamo gli indici della classe da sbilanciare nel TRAIN (1/10)

In [ ]:
#TRAIN
rows2remove_tr1 = np.random.choice((y_train1), 1374 - 137, replace=False) #1/10
#rows2remove_tr1

In [ ]:
rows2remove_tr2 = np.random.choice((y_train2), 1073 - 107, replace=False) #1/10

In [ ]:
#unisco i due array in modo da avere un unico array con gli indici delle classi da levare
rows2remove_tr=np.concatenate((rows2remove_tr1, rows2remove_tr2), axis=None)

In [ ]:
rows2remove_tr

### Eliminiamo gli indici delle classi da sbilanciare nel TEST (1/10)

In [ ]:
#TEST
rows2remove_tes1 = np.random.choice((y_test1), 532 - 53, replace=False) # 1/10

In [ ]:
rows2remove_tes2 = np.random.choice((y_test2), 471 - 47, replace=False) # 1/10

In [ ]:
rows2remove_tes = np.concatenate((rows2remove_tes1, rows2remove_tes2), axis=None)

### Osserviamo com'è cambiata la distribuzione

In [ ]:
X_test_sbilanciato = X_test.drop(rows2remove_tes, axis=0)
y_test_sbilanciato = y_test.drop(rows2remove_tes, axis=0)

X_train_sbilanciato = X_train.drop(rows2remove_tr, axis=0)
y_train_sbilanciato = y_train.drop(rows2remove_tr, axis=0)



In [ ]:
#Originale
np.unique(y_train, return_counts=True), np.unique(y_test, return_counts=True)

In [ ]:
#Sbilanciato
np.unique(y_train_sbilanciato, return_counts=True), np.unique(y_test_sbilanciato, return_counts=True)

## Classification

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score 
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV


### DecisionTree

In [ ]:
param_list = {'max_depth': [None],    
             'min_samples_split': [2, 4, 5, 10, 20, 30],
             'min_samples_leaf': [1, 4, 5, 10, 20, 30], 
             'criterion': [ 'entropy']
             }

In [ ]:
clf_search = DecisionTreeClassifier() 
clf_search.fit(X_train_sbilanciato, y_train_sbilanciato)

In [ ]:
grid_search = GridSearchCV(clf_search, param_grid=param_list)
grid_search.fit(X_train_sbilanciato, y_train_sbilanciato) ##### ASK !!
res = grid_search.cv_results_

In [ ]:
grid_search.best_params_

In [ ]:
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train_sbilanciato, y_train_sbilanciato)
y_pred0 = clf.predict(X_test_sbilanciato)

print('Accuracy %s' % accuracy_score(y_test_sbilanciato, y_pred0))
print('F1-score %s' % f1_score(y_test_sbilanciato, y_pred0, average=None))
print(classification_report(y_test_sbilanciato, y_pred0))

### KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

error_rate = []
for i in range(1,30):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train_sbilanciato,y_train_sbilanciato.values.ravel())
    pred_i = knn.predict(X_test_sbilanciato)
    error_rate.append(np.mean(pred_i != np.ravel(y_test_sbilanciato)))
#print(pred_i)
plt.figure(figsize=(10,6))
plt.plot(range(1,30),error_rate,color='blue', linestyle='dashed', 
         marker='o',markerfacecolor='red', markersize=10)
plt.title('Error Rate vs. K Value')
plt.xlabel('K')
plt.ylabel('Error Rate')
print("Minimum error:-",min(error_rate),"at K =",error_rate.index(min(error_rate)))

In [ ]:
clf_knn = KNeighborsClassifier(n_neighbors=17, weights='uniform') # 26 vicini
clf_knn.fit(X_train_sbilanciato, np.ravel(y_train_sbilanciato)) #df3
y_pred_knn = clf_knn.predict(X_test_sbilanciato)

In [ ]:
print('Accuracy %s' % accuracy_score(y_test_sbilanciato, y_pred_knn))
print('F1-score %s' % f1_score(y_test_sbilanciato, y_pred_knn, average=None))
print(classification_report(y_test_sbilanciato, y_pred_knn))

## OutOfProject

### RandomUnderSampler

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import CondensedNearestNeighbour

In [ ]:
rus = RandomUnderSampler(random_state=42)
X_res, y_res = rus.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_res))

In [ ]:
pca = PCA(n_components=2)
pca.fit(X_train)
X_pca = pca.transform(X_res)

plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y_res, cmap=plt.cm.prism, edgecolor='k', alpha=0.7)
plt.show()

In [ ]:
clf = DecisionTreeClassifier(min_samples_leaf=3, random_state=42)
clf.fit(X_res, y_res)

y_pred = clf.predict(X_test)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

y_score = clf.predict_proba(X_test)
fpr, tpr, _ = roc_curve(y_test, y_score[:, 1])
roc_auc = auc(fpr, tpr)

plt.plot(fpr0, tpr0, color='darkorange', lw=3, label='$AUC_0$ = %.3f' % (roc_auc0))
plt.plot(fpr, tpr, color='green', lw=3, label='$AUC_1$ = %.3f' % (roc_auc))

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=16)
plt.ylabel('True Positive Rate', fontsize=16)
plt.title('ROC curve', fontsize=16)
plt.legend(loc="lower right", fontsize=14, frameon=False)
plt.tick_params(axis='both', which='major', labelsize=16)
plt.show()

### CondensedNearestNeighbour

In [ ]:
import warnings
warnings.simplefilter("ignore")

In [ ]:
cnn = CondensedNearestNeighbour(random_state=42, n_jobs=10)
X_res, y_res = cnn.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_res))

In [ ]:
pca = PCA(n_components=2)
pca.fit(X_train)
X_pca = pca.transform(X_res)

plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y_res, cmap=plt.cm.prism, edgecolor='k', alpha=0.7)
plt.show()

In [ ]:
clf = DecisionTreeClassifier(min_samples_leaf=3, random_state=42)
clf.fit(X_res, y_res)

y_pred = clf.predict(X_test)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

y_score = clf.predict_proba(X_test)
fpr, tpr, _ = roc_curve(y_test, y_score[:, 1])
roc_auc = auc(fpr, tpr)

plt.plot(fpr0, tpr0, color='darkorange', lw=3, label='$AUC_0$ = %.3f' % (roc_auc0))
plt.plot(fpr, tpr, color='green', lw=3, label='$AUC_1$ = %.3f' % (roc_auc))

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=16)
plt.ylabel('True Positive Rate', fontsize=16)
plt.title('ROC curve', fontsize=16)
plt.legend(loc="lower right", fontsize=14, frameon=False)
plt.tick_params(axis='both', which='major', labelsize=16)
plt.show()

## Oversampling

In [ ]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

### RandomOverSampler

In [ ]:
ros = RandomOverSampler(random_state=42)
X_res, y_res = ros.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_res))

In [ ]:
pca = PCA(n_components=2)
pca.fit(X_train)
X_pca = pca.transform(X_res)

plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y_res, cmap=plt.cm.prism, edgecolor='k', alpha=0.7)
plt.show()

In [ ]:
clf = DecisionTreeClassifier(min_samples_leaf=3, random_state=42)
clf.fit(X_res, y_res)

y_pred = clf.predict(X_test)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

y_score = clf.predict_proba(X_test)
fpr, tpr, _ = roc_curve(y_test, y_score[:, 1])
roc_auc = auc(fpr, tpr)

plt.plot(fpr0, tpr0, color='darkorange', lw=3, label='$AUC_0$ = %.3f' % (roc_auc0))
plt.plot(fpr, tpr, color='green', lw=3, label='$AUC_1$ = %.3f' % (roc_auc))

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=16)
plt.ylabel('True Positive Rate', fontsize=16)
plt.title('ROC curve', fontsize=16)
plt.legend(loc="lower right", fontsize=14, frameon=False)
plt.tick_params(axis='both', which='major', labelsize=16)
plt.show()

### SMOTE

In [ ]:
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_res))

In [ ]:
pca = PCA(n_components=2)
pca.fit(X_train)
X_pca = pca.transform(X_res)

plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y_res, cmap=plt.cm.prism, edgecolor='k', alpha=0.7)
plt.show()

In [ ]:
clf = DecisionTreeClassifier(min_samples_leaf=3, random_state=42)
clf.fit(X_res, y_res)

y_pred = clf.predict(X_test)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

y_score = clf.predict_proba(X_test)
fpr, tpr, _ = roc_curve(y_test, y_score[:, 1])
roc_auc = auc(fpr, tpr)

plt.plot(fpr0, tpr0, color='darkorange', lw=3, label='$AUC_0$ = %.3f' % (roc_auc0))
plt.plot(fpr, tpr, color='green', lw=3, label='$AUC_1$ = %.3f' % (roc_auc))

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=16)
plt.ylabel('True Positive Rate', fontsize=16)
plt.title('ROC curve', fontsize=16)
plt.legend(loc="lower right", fontsize=14, frameon=False)
plt.tick_params(axis='both', which='major', labelsize=16)
plt.show()